### About

This is the notebook for building PredictionByHero model.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
#load and randomise data
dataset = pd.read_csv('heroSelect.csv', index_col = 0)
dataset = dataset.take(np.random.permutation(len(dataset)))
dataset.head(10)

,team1Win,t1_1,t1_10,t1_101,t1_102,t1_103,t1_104,t1_105,t1_106,t1_107,...,t2_85,t2_86,t2_89,t2_9,t2_90,t2_91,t2_92,t2_96,t2_98,t2_99
95892,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
135581,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
60975,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
26646,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
133570,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
100498,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
45465,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
95029,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
128637,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0
53080,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0.0,0,0,0


In [3]:
print('dataset', dataset.shape)
dataset, validation = train_test_split(dataset, test_size = 0.1)
train, test = train_test_split(dataset, test_size = 0.1)
print('train:', train.shape, 'validation:', validation.shape, 'test:', test.shape)

dataset (183452, 273)
train: (148595, 273) validation: (18346, 273) test: (16511, 273)


## Tensorflow parts:

In [4]:
sess = tf.InteractiveSession()

In [5]:
#input/output placeholders
x_team1    = tf.placeholder("float", shape=[None, 136], name='x_dire')
x_team2 = tf.placeholder("float", shape=[None, 136], name='x_radiant')
y_true = tf.placeholder("float", shape=[None, 2], name='y_true')

#we'll use dropout layers for regularisation which need a keep probability
keep_prob1 = tf.placeholder("float", name='keep_prob1')
keep_prob2 = tf.placeholder("float", name='keep_prob2')

#there doesn't seem to be any other way to differenciate train and validation summaries for TensorBoard
loss_name     = tf.placeholder("string", name='loss_name')
accuracy_name = tf.placeholder("string", name='accuracy_name')

#### Weight init for fully connected layer:

In [6]:
def fc_weight_bias(in_size, out_size):
    initial_weight = tf.truncated_normal([in_size, out_size], stddev=0.2, mean=0.0)
    initial_bias = tf.constant(0.1, shape=[out_size])
    return tf.Variable(initial_weight), tf.Variable(initial_bias)

#### Our model structure

In [7]:
with tf.name_scope("hero_layers_1") as scope:
    W_hero1, b_hero1 = fc_weight_bias(136,80)      
    #note that team1 layer and team2 layer use the same weights and biases
    team1_layer1 = tf.nn.relu(tf.matmul(x_team1, W_hero1) + b_hero1)
    team2_layer1 = tf.nn.relu(tf.matmul(x_team2, W_hero1) + b_hero1)

#second hero layer
with tf.name_scope("hero_layers_2") as scope:    
    W_hero2, b_hero2 = fc_weight_bias(80,80)    
    #again, team1 and team2 use the same weights and biases
    team1_layer2 = tf.nn.relu(tf.matmul(team1_layer1, W_hero2) + b_hero2)
    team2_layer2 = tf.nn.relu(tf.matmul(team2_layer1, W_hero2) + b_hero2)

#now concatenate the team1 and team2 team outputs
with tf.name_scope("hero_layers_concat") as scope:
    team1_team2_concat = tf.concat([team1_layer2, team2_layer2], 1)
    team1_team2_drop = tf.nn.dropout(team1_team2_concat, keep_prob1)
    h_drop1 = tf.nn.dropout(team1_team2_drop, keep_prob1)

with tf.name_scope("hidden_layer_1") as scope:
    W_hidden1, b_hidden1 = fc_weight_bias(160,120)    
    h_hidden1 = tf.nn.relu(tf.matmul(h_drop1, W_hidden1) + b_hidden1)
    h_drop2 = tf.nn.dropout(h_hidden1, keep_prob2)

with tf.name_scope("hidden_layer_2") as scope:
    W_hidden2, b_hidden2 = fc_weight_bias(120,75)    
    h_hidden2 = tf.nn.relu(tf.matmul(h_drop2, W_hidden2) + b_hidden2)

with tf.name_scope("hidden_layer_3") as scope:
    W_hidden3, b_hidden3 = fc_weight_bias(75,25)    
    h_hidden3 = tf.nn.relu(tf.matmul(h_hidden2, W_hidden3) + b_hidden3)

with tf.name_scope("output_layer") as scope:
    W_hidden4, b_hidden4 = fc_weight_bias(25,2)    
    y_pred = tf.nn.softmax(tf.matmul(h_hidden3, W_hidden4) + b_hidden4)

In [8]:
with tf.name_scope("loss_calculations") as scope:
    cross_entropy = -tf.reduce_sum(y_true * tf.log(y_pred + 1e-8))
    weights_sum   = tf.add_n([tf.nn.l2_loss(variable) for variable in tf.global_variables()])
    mean_loss     = cross_entropy + weights_sum
    loss          = tf.reduce_mean(mean_loss)

with tf.name_scope("trainer") as scope:
    train_step    = tf.train.AdamOptimizer(0.0001).minimize(loss)

with tf.name_scope("accuracy_calculations") as scope:
    correct  = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, "float"))

In [9]:
sess.run(tf.global_variables_initializer())

#### DataFeed Function

In [10]:
def get_data_feed(dataset, kp1=1.0, kp2=1.0, loss_str='loss', accuracy_str='accuracy'):
    team1_data, team2_data = dataset.ix[:,1:137], dataset.ix[:,137:273]
    winners = pd.get_dummies(dataset['team1Win'])
    return {
        x_team1: team1_data,
        x_team2: team2_data,
        y_true: winners,
        loss_name: loss_str,
        accuracy_name: accuracy_str,
        keep_prob1: kp1,
        keep_prob2: kp2
    }  

In [11]:
train_feed      = get_data_feed(train,      loss_str = 'loss_train',      accuracy_str = 'accuracy_train')
validation_feed = get_data_feed(validation, loss_str = 'loss_validation', accuracy_str = 'accuracy_validation')
test_feed       = get_data_feed(test,       loss_str = 'loss_test',       accuracy_str = 'accuracy_test')

C:\Users\tim12\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


#### Batch Function

In [12]:
def get_batches(dataset, batch_size=1500): #1500 is about 1% of the entire training sets
    #randomise before every epoch
    dataset = dataset.take(np.random.permutation(len(dataset)))
    
    i = 0
    while i < len(dataset):
        yield dataset[i : i + batch_size]
        i = i + batch_size 

In [13]:
for i in range(1000):    
    for mini_batch in get_batches(train):
        mini_batch_feed = get_data_feed(mini_batch, 0.5, 0.5)   
        train_step.run(feed_dict = mini_batch_feed)
    
    #log every epoch
    train_loss          = loss.eval(feed_dict = train_feed)
    validation_loss     = loss.eval(feed_dict = validation_feed)

    train_accuracy      = accuracy.eval(feed_dict = train_feed)
    validation_accuracy = accuracy.eval(feed_dict = validation_feed)

    print("epoch %d, loss: %g, train: %g, validation: %g"% (i, train_loss, train_accuracy, validation_accuracy)) 


C:\Users\tim12\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


epoch 0, loss: 122231, train: 0.511141, validation: 0.51052
epoch 1, loss: 110933, train: 0.510953, validation: 0.511719
epoch 2, loss: 106668, train: 0.510495, validation: 0.512264
epoch 3, loss: 105162, train: 0.508644, validation: 0.509811
epoch 4, loss: 104429, train: 0.507493, validation: 0.512319
epoch 5, loss: 104167, train: 0.506679, validation: 0.508503
epoch 6, loss: 104018, train: 0.50538, validation: 0.508503
epoch 7, loss: 103928, train: 0.504687, validation: 0.508776
epoch 8, loss: 103879, train: 0.503476, validation: 0.506595
epoch 9, loss: 103833, train: 0.503752, validation: 0.507468
epoch 10, loss: 103802, train: 0.504108, validation: 0.507631
epoch 11, loss: 103782, train: 0.503335, validation: 0.508776
epoch 12, loss: 103759, train: 0.503092, validation: 0.508558
epoch 13, loss: 103731, train: 0.503462, validation: 0.507522
epoch 14, loss: 103713, train: 0.50322, validation: 0.507413
epoch 15, loss: 103691, train: 0.503826, validation: 0.509103
epoch 16, loss: 10366

epoch 133, loss: 102452, train: 0.536438, validation: 0.52769
epoch 134, loss: 102435, train: 0.537205, validation: 0.529107
epoch 135, loss: 102438, train: 0.537575, validation: 0.532977
epoch 136, loss: 102427, train: 0.536324, validation: 0.528235
epoch 137, loss: 102423, train: 0.537266, validation: 0.528072
epoch 138, loss: 102421, train: 0.537367, validation: 0.528562
epoch 139, loss: 102411, train: 0.537394, validation: 0.530034
epoch 140, loss: 102417, train: 0.536882, validation: 0.527853
epoch 141, loss: 102415, train: 0.538766, validation: 0.530906
epoch 142, loss: 102409, train: 0.539507, validation: 0.53325
epoch 143, loss: 102397, train: 0.538396, validation: 0.531015
epoch 144, loss: 102396, train: 0.538955, validation: 0.532596
epoch 145, loss: 102387, train: 0.539406, validation: 0.532378
epoch 146, loss: 102391, train: 0.539534, validation: 0.532105
epoch 147, loss: 102376, train: 0.539345, validation: 0.531233
epoch 148, loss: 102363, train: 0.539399, validation: 0.5

ResourceExhaustedError: OOM when allocating tensor with shape[148595,80]
	 [[Node: hero_layers_1/MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_x_radiant_0_5, hero_layers_1/Variable/read)]]

Caused by op 'hero_layers_1/MatMul_1', defined at:
  File "C:\Users\tim12\.conda\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\tim12\.conda\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\tim12\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\tim12\.conda\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\tim12\.conda\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\tim12\.c

In [ ]:
accuracy.eval(feed_dict=test_feed)